<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-03 10:00:00
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: 2.40 L (1.76%)
Current PnL: -30.67 L (-19.56%)
CY Booked + Current PnL: -15.93 L (-10.16%)
-------------------
Total profit:  1.36 L
Total loss:  -32.03 L
-------------------
Total Booked + Current PnL: 11.05 L (8.51%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.62%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 96.18 L (69.28%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.45,3.68,15.00,19.23,23399.0,5532.0,155992.0,484.83,21.35,50.0,L-LC,12.45,182.0,0.24,1.12,25.73,XY25,ATH,MINING
50,MASFIN,1.88,-3.66,26.68,22.05,25185.0,-3585.0,94395.0,398.61,-16.89,50.0,H-SC,3.47,168.0,-0.14,0.68,37.21,XR,ATH,FINANCE
77,TTKPRESTIG,0.53,-27.08,37.25,0.09,27375.0,-27286.0,73491.0,770.00,70.54,35.0,M-SC,7.12,253.0,-1.00,0.53,0.53,OX40N,NTT,DURABLES
33,HCLTECH,2.13,11.14,11.42,23.83,30706.0,26954.0,268878.0,1908.19,16.15,59.0,X-LC,7.68,13.0,0.88,1.94,29.00,X40,ATH,IT
51,MEDANTA,-1.82,-6.53,32.75,24.08,38860.0,-8294.0,118656.0,1486.00,-14.15,49.0,X-SC,10.29,91.0,-0.21,0.85,8.68,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,11.76,-22.96,108.68,60.77,153037.0,-41961.0,140814.0,521.00,62.53,65.0,H-SC,4.45,158.0,-0.27,1.01,27.66,XR,NTT,JEWELLERY
76,TRIDENT,10.78,-23.51,68.01,28.51,47993.0,-21686.0,70568.0,48.00,-9.54,66.0,M-SC,10.77,226.0,-0.45,0.51,20.75,XR,NTT,TEXTILES
67,SONACOMS,6.74,-7.77,50.81,39.09,47406.0,-7858.0,93301.0,804.02,-27.83,74.0,M-SC,2.34,227.0,-0.17,0.67,31.99,AR,ATH,AUTO
23,DIXON,6.72,-23.50,71.58,31.26,126370.0,-54224.0,176544.0,18931.72,-64.29,45.0,X-MC,27.77,56.0,-0.43,1.27,8.35,X40N,ATH,IT
5,ANGELONE,6.31,4.54,18.81,24.20,60025.0,13862.0,319112.0,3033.00,48.84,52.0,X-SC,2.02,99.0,0.23,2.30,33.65,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,-4.09,-21.48,58.54,24.48,122891.0,-57439.0,209926.0,387.0,-22.78,19.0,X-LC,7.59,37.0,-0.47,1.51,21.68,XY24,BTT,FINANCE
45,JCHAC,-3.77,-42.19,72.96,-0.01,53907.0,-53919.0,73886.0,2282.0,15030.43,39.0,M-SC,15.50,235.0,-1.00,0.53,2.44,OX40N,BTT,AC
51,MEDANTA,-1.82,-6.53,32.75,24.08,38860.0,-8294.0,118656.0,1486.0,-14.15,49.0,X-SC,10.29,91.0,-0.21,0.85,8.68,XY24,NTT,HEALTHCARE
48,KANSAINER,-1.78,-23.24,47.82,13.47,98992.0,-62658.0,207009.0,340.0,-68.51,49.0,H-SC,5.11,159.0,-0.63,1.49,5.26,XY24,NTT,PAINTS
15,CAMPUS,-1.67,-9.19,46.78,33.28,71269.0,-15425.0,152350.0,393.0,-26.89,59.0,M-SC,5.56,221.0,-0.22,1.10,18.08,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,1.49,-0.24,21.03,20.74,42623.0,-495.0,202675.0,446.0,31.15,52.0,M-MC,4.56,189.0,-0.01,1.46,10.35,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.57,-6.09,121.40,107.91,168322.0,-8993.0,138651.0,1641.55,-23.39,49.0,M-SC,11.26,220.0,-0.05,1.00,4.31,OX40N,ATH,CABLES
72,TATAELXSI,1.29,-20.59,66.26,32.03,69368.0,-27145.0,104690.0,9161.00,-14.02,54.0,H-SC,6.50,157.0,-0.39,0.75,17.26,OX40N,NTT,IT
66,SIS,-0.09,-22.34,56.14,21.26,48408.0,-24804.0,86228.0,528.00,2036.79,57.0,H-SC,4.98,163.0,-0.51,0.62,16.50,OX40N,NTT,MISC
48,KANSAINER,-1.78,-23.24,47.82,13.47,98992.0,-62658.0,207009.0,340.00,-68.51,49.0,H-SC,5.11,159.0,-0.63,1.49,5.26,XY24,NTT,PAINTS
40,INDIGOPNTS,1.58,-25.50,34.26,0.02,44551.0,-44520.0,130039.0,1408.00,98.10,35.0,M-SC,5.27,240.0,-1.00,0.94,13.11,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.45,3.68,15.00,19.23,23399.0,5532.0,155992.0,484.83,21.35,50.0,L-LC,12.45,182.0,0.24,1.12,25.73,XY25,ATH,MINING
84,WIPRO,1.85,1.22,70.19,72.27,120384.0,2064.0,171512.0,420.00,-13.34,51.0,M-LC,2.64,101.0,0.02,1.24,10.24,XR,NTT,IT
1,ABB,2.83,3.17,41.10,45.57,110931.0,8285.0,269904.0,7934.00,-33.65,69.0,H-MC,6.11,121.0,0.07,1.94,19.86,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,2.83,3.17,41.10,45.57,110931.0,8285.0,269904.0,7934.00,-33.65,69.0,H-MC,6.11,121.0,0.07,1.94,19.86,AR,NTT,ELECTRICAL
84,WIPRO,1.85,1.22,70.19,72.27,120384.0,2064.0,171512.0,420.00,-13.34,51.0,M-LC,2.64,101.0,0.02,1.24,10.24,XR,NTT,IT
39,INDIAMART,1.97,-3.16,113.46,106.72,135518.0,-3895.0,119441.0,4810.62,-51.96,55.0,H-SC,8.94,138.0,-0.03,0.86,19.48,AR,ATH,MISC
26,FINCABLES,2.57,-6.09,121.40,107.91,168322.0,-8993.0,138651.0,1641.55,-23.39,49.0,M-SC,11.26,220.0,-0.05,1.00,4.31,OX40N,ATH,CABLES
85,ZYDUSLIFE,2.57,-4.85,42.54,35.63,84828.0,-10166.0,199408.0,1286.17,-18.07,52.0,H-MC,5.81,119.0,-0.12,1.44,11.01,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,-4.09,-21.48,58.54,24.48,122891.0,-57439.0,209926.0,387.00,-22.78,19.0,X-LC,7.59,37.0,-0.47,1.51,21.68,XY24,BTT,FINANCE
44,ITC,-0.49,-23.25,44.34,10.78,81783.0,-55873.0,184445.0,452.00,-54.23,20.0,X-LC,18.43,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
13,BERGEPAINT,1.29,-15.89,44.02,21.14,84175.0,-36118.0,191221.0,680.00,-21.90,28.0,X-MC,7.78,75.0,-0.43,1.38,2.00,XY24,NTT,PAINTS
53,PGHH,-0.42,-12.04,52.56,34.19,98712.0,-25712.0,187808.0,17907.65,-38.15,28.0,X-MC,8.09,60.0,-0.26,1.35,0.00,X40,ATH,FMCG
6,ASIANPAINT,1.66,-14.65,41.71,20.95,89626.0,-36890.0,214878.0,3460.25,-15.53,29.0,X-LC,10.23,36.0,-0.41,1.55,15.76,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,0.75,-12.19,35.01,18.55,116277.0,-46113.0,332124.0,4311.59,-24.27,54.0,X-LC,0.59,3.0,-0.40,2.39,12.99,X40,ATH,IT
81,VBL,2.66,-3.30,40.30,35.67,123081.0,-10431.0,305411.0,671.64,-14.83,48.0,X-LC,1.52,5.0,-0.08,2.20,10.09,X40N,ATH,FMCG
5,ANGELONE,6.31,4.54,18.81,24.20,60025.0,13862.0,319112.0,3033.00,48.84,52.0,X-SC,2.02,99.0,0.23,2.30,33.65,X40N,NTT,FINANCE
34,HINDUNILVR,-0.12,-8.41,24.11,13.67,56765.0,-21623.0,235440.0,2922.00,-15.28,47.0,X-LC,2.28,9.0,-0.38,1.70,11.15,XY25,NTT,FMCG
21,DABUR,-0.07,-2.51,47.40,43.70,114635.0,-6223.0,241845.0,735.00,-10.52,40.0,X-MC,2.73,73.0,-0.05,1.74,12.16,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,-0.42,-12.04,52.56,34.19,98712.0,-25712.0,187808.0,17907.65,-38.15,28.0,X-MC,8.09,60.0,-0.26,1.35,0.00,X40,ATH,FMCG
44,ITC,-0.49,-23.25,44.34,10.78,81783.0,-55873.0,184445.0,452.00,-54.23,20.0,X-LC,18.43,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
3,ACC,1.29,-30.36,135.71,64.15,224885.0,-72241.0,165710.0,3906.00,-59.37,38.0,X-SC,7.63,84.0,-0.32,1.19,1.29,XY24,BTT,CEMENT
13,BERGEPAINT,1.29,-15.89,44.02,21.14,84175.0,-36118.0,191221.0,680.00,-21.90,28.0,X-MC,7.78,75.0,-0.43,1.38,2.00,XY24,NTT,PAINTS
12,BATAINDIA,0.78,-43.86,140.84,35.20,101736.0,-56435.0,72235.0,2096.00,-5.27,32.0,X-SC,20.41,93.0,-0.55,0.52,2.65,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.07,-28.83,100.71,42.84,46592.0,-18742.0,46264.0,424.00,-53.17,55.0,X-SC,21.68,83.0,-0.40,0.33,8.75,XY24,NTT,MISC
12,BATAINDIA,0.78,-43.86,140.84,35.20,101736.0,-56435.0,72235.0,2096.00,-5.27,32.0,X-SC,20.41,93.0,-0.55,0.52,2.65,X40,NTT,FOOTWEAR
58,RELAXO,1.48,-49.80,203.41,52.31,148223.0,-72291.0,72869.0,1176.00,-46.57,51.0,X-SC,11.60,92.0,-0.49,0.53,8.36,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.82,-6.53,32.75,24.08,38860.0,-8294.0,118656.0,1486.00,-14.15,49.0,X-SC,10.29,91.0,-0.21,0.85,8.68,XY24,NTT,HEALTHCARE
35,HONAUT,0.95,-16.67,71.99,43.32,97705.0,-27152.0,135720.0,58357.33,-28.43,56.0,X-SC,8.80,90.0,-0.28,0.98,10.20,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.49,-23.25,44.34,10.78,81783.0,-55873.0,184445.0,452.00,-54.23,20.0,X-LC,18.43,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
75,TMPV,1.80,-24.28,62.40,22.98,106738.0,-54843.0,171055.0,600.00,-21.86,63.0,X-LC,4.46,2.0,-0.51,1.23,9.35,XY24,NTT,AUTO
73,TCS,0.75,-12.19,35.01,18.55,116277.0,-46113.0,332124.0,4311.59,-24.27,54.0,X-LC,0.59,3.0,-0.40,2.39,12.99,X40,ATH,IT
81,VBL,2.66,-3.30,40.30,35.67,123081.0,-10431.0,305411.0,671.64,-14.83,48.0,X-LC,1.52,5.0,-0.08,2.20,10.09,X40N,ATH,FMCG
42,INFY,2.07,13.22,18.57,34.24,65782.0,41352.0,354240.0,1972.00,-11.70,56.0,X-LC,7.23,6.0,0.63,2.55,22.59,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1.02,-32.11,95.76,32.89,50190.0,-24794.0,52412.0,1800.00,-747.22,59.0,L-MC,11.20,259.0,-0.49,0.38,44.34,XR,NTT,BANKS
67,SONACOMS,6.74,-7.77,50.81,39.09,47406.0,-7858.0,93301.0,804.02,-27.83,74.0,M-SC,2.34,227.0,-0.17,0.67,31.99,AR,ATH,AUTO
14,BSOFT,4.57,-19.32,86.90,50.79,98610.0,-27178.0,113475.0,831.70,3.59,61.0,H-SC,6.15,171.0,-0.28,0.82,32.12,XR,ATH,IT
79,VAIBHAVGBL,11.76,-22.96,108.68,60.77,153037.0,-41961.0,140814.0,521.00,62.53,65.0,H-SC,4.45,158.0,-0.27,1.01,27.66,XR,NTT,JEWELLERY
18,COALINDIA,-0.45,3.68,15.00,19.23,23399.0,5532.0,155992.0,484.83,21.35,50.0,L-LC,12.45,182.0,0.24,1.12,25.73,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,6.74,-7.77,50.81,39.09,47406.0,-7858.0,93301.0,804.02,-27.83,74.0,M-SC,2.34,227.0,-0.17,0.67,31.99,AR,ATH,AUTO
1,ABB,2.83,3.17,41.10,45.57,110931.0,8285.0,269904.0,7934.00,-33.65,69.0,H-MC,6.11,121.0,0.07,1.94,19.86,AR,NTT,ELECTRICAL
62,SATIN,1.94,-13.01,74.53,51.83,179134.0,-35932.0,240352.0,274.00,-22.47,61.0,H-SC,2.96,148.0,-0.20,1.73,17.09,XY24,NTT,FINANCE
79,VAIBHAVGBL,11.76,-22.96,108.68,60.77,153037.0,-41961.0,140814.0,521.00,62.53,65.0,H-SC,4.45,158.0,-0.27,1.01,27.66,XR,NTT,JEWELLERY
33,HCLTECH,2.13,11.14,11.42,23.83,30706.0,26954.0,268878.0,1908.19,16.15,59.0,X-LC,7.68,13.0,0.88,1.94,29.00,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.84
1,25,44.69
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.20
MC    30.55
LC    24.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.16
X40      23.86
X40N     15.09
AR        8.97
XR        8.90
XY25      8.62
OX40N     7.42
SR        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.88
H-SC    23.04
X-LC    20.68
M-SC    11.12
X-SC    10.30
H-MC     4.83
M-MC     1.46
M-LC     1.24
H-LC     1.16
L-LC     1.12
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.98,-13.91,52.74
FINANCE,13.17,-14.23,59.20
IT,12.97,-18.70,78.00
MISC,6.96,-34.19,89.08
ELECTRICAL,6.09,-11.12,51.66
PAINTS,5.36,-24.25,42.70
INSURANCE,4.83,-1.14,35.76
PHARMA,4.03,-4.98,38.03
AUTO,2.95,-27.70,71.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3337441.0,21
AR,1325273.0,10
XR,1258471.0,12
X40,1195929.0,15
X40N,1026611.0,10
OX40N,760159.0,10
XY25,423010.0,6
SR,290874.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3593389.0,24
X-MC,1715827.0,16
M-SC,1514943.0,15
X-LC,1020122.0,12
X-SC,867981.0,9
H-MC,408644.0,3
L-SC,187446.0,2
M-LC,120384.0,1
H-LC,72820.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1235034.0      6
           AR         934745.0      5
M-SC       XY24       865120.0      6
H-SC       XR         808969.0      7
X-MC       X40        573488.0      7
           XY24       484436.0      3
X-LC       X40        445285.0      6
X-MC       X40N       423419.0      4
X-SC       X40N       380488.0      4
M-SC       OX40N      344041.0      5
H-SC       OX40N      323767.0      4
X-SC       XY24       310337.0      3
H-SC       SR         290874.0      2
X-MC       XY25       234484.0      2
X-LC       XY24       229629.0      2
           X40N       222704.0      2
H-MC       XY24       212885.0      1
           AR         195759.0      2
M-SC       XR         183833.0      2
X-SC       X40        177156.0      2
X-LC       XY25       122504.0      2
M-SC       AR         121949.0      2
M-LC       XR         120384.0      1
L-SC       XR          95095.0      1
           OX40N       92351.0      1
H-LC       AR          72820.0      1
L-MC       XR          50190.0      1
M-MC       XY25        42623.0      1
L-LC       XY25        23399.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
